In [5]:
SHARE_API = 'https://staging-share.osf.io/api/search/abstractcreativework/_search'
PROD_SHARE_API= 'https://share.osf.io/api/search/abstractcreativework/_search'

In [6]:
import furl
import requests

search_url = furl.furl(PROD_SHARE_API)
search_url.args['size'] = 3
recent_results = requests.get(search_url.url).json()

recent_results = recent_results['hits']['hits']

print('The request URL is {}'.format(search_url.url))
print('----------')
for result in recent_results:
    print(
        '{} -- from {}'.format(
            result['_source']['title'],
            result['_source']['sources']
        )
    )

The request URL is https://share.osf.io/api/search/abstractcreativework/_search?size=3
----------
SH078169.07FU
Rhizophagus irregularis (B&#322;aszk., Wubet, Renker & Buscot) C. Walker & A. Schüßler -- from ['DataCite MDS']
CCDC 1462836: Experimental Crystal Structure Determination -- from ['DataCite MDS']
GBIF Occurrence Download -- from ['DataCite MDS']
ТЕОРЕТИЧЕСКИЕ И ПРАКТИЧЕСКИЕ АСПЕКТЫ ПРИМЕНЕНИЯ НАНОТЕХНОЛОГИЙ ПРИ ПОДГОТОВКЕ СЕМЯН К ПОСЕВУ -- from ['CyberLeninka - Russian open access scientific library']
ВНУТРИВУЗОВСКОЕ ЭЛЕКТРОННОЕ ПРОСТРАНСТВО НА ОСНОВЕ НОВЫХ ОБРАЗОВАТЕЛЬНЫХ ТЕХНОЛОГИЙ -- from ['CyberLeninka - Russian open access scientific library']
ХИМИЧЕСКИЙ СОСТАВ ДРЕВЕСИНЫ ДУБА, ИСПОЛЬЗУЕМОЙ ДЛЯ ПРОИЗВОДСТВА КОНЬЯКА И БРЕНДИ -- from ['CyberLeninka - Russian open access scientific library']
SH082013.07FU
Trichaptum abietinum (Dicks.) Ryvarden -- from ['DataCite MDS']
МЕТОД ОПРЕДЕЛЕНИЯ ВЫХОДА КОРОТКИХ ЗАГОТОВОК ИЗ ПИЛОМАТЕРИАЛОВ -- from ['CyberLeninka - Russian open access s

In [7]:
search_url.args['q'] = 'university of oregon'
recent_results = requests.get(search_url.url).json()

recent_results = recent_results['hits']['hits']

print('The request URL is {}'.format(search_url.url))
print('---------')
for result in recent_results:
    print(
        '{} -- from {}'.format(
            result['_source']['title'],
            result['_source']['sources']
        )
    )

The request URL is https://share.osf.io/api/search/abstractcreativework/_search?size=3&q=university+of+oregon
---------
SH078169.07FU
Rhizophagus irregularis (B&#322;aszk., Wubet, Renker & Buscot) C. Walker & A. Schüßler -- from ['DataCite MDS']
CCDC 1462836: Experimental Crystal Structure Determination -- from ['DataCite MDS']
GBIF Occurrence Download -- from ['DataCite MDS']
ТЕОРЕТИЧЕСКИЕ И ПРАКТИЧЕСКИЕ АСПЕКТЫ ПРИМЕНЕНИЯ НАНОТЕХНОЛОГИЙ ПРИ ПОДГОТОВКЕ СЕМЯН К ПОСЕВУ -- from ['CyberLeninka - Russian open access scientific library']
ВНУТРИВУЗОВСКОЕ ЭЛЕКТРОННОЕ ПРОСТРАНСТВО НА ОСНОВЕ НОВЫХ ОБРАЗОВАТЕЛЬНЫХ ТЕХНОЛОГИЙ -- from ['CyberLeninka - Russian open access scientific library']
ХИМИЧЕСКИЙ СОСТАВ ДРЕВЕСИНЫ ДУБА, ИСПОЛЬЗУЕМОЙ ДЛЯ ПРОИЗВОДСТВА КОНЬЯКА И БРЕНДИ -- from ['CyberLeninka - Russian open access scientific library']
SH082013.07FU
Trichaptum abietinum (Dicks.) Ryvarden -- from ['DataCite MDS']
МЕТОД ОПРЕДЕЛЕНИЯ ВЫХОДА КОРОТКИХ ЗАГОТОВОК ИЗ ПИЛОМАТЕРИАЛОВ -- from ['CyberLeninka -

In [8]:
affiliation_query = {
    "query": {
        "bool": {
            "must": {
                "query_string": {
                    "query": "*"
                }
            },
            "filter": [
                {
                    "term": {
                        "institutions.raw": "University of Oregon Libraries"
                    }
                }
            ]
        }
    }
}

In [9]:
import json
import requests

def query_share(url, query):
    # A helper function that will use the requests library,
    # pass along the correct headers,
    # and make the query we want
    headers = {'Content-Type': 'application/json'}
    data = json.dumps(query)
    return requests.post(url, headers=headers, data=data, verify=False).json()

In [10]:
#with_tags = query_share(search_url.url, tags_query)
#missing_tags = query_share(search_url.url, missing_tags_query)
affiliation_results = query_share(search_url.url, affiliation_query)
print(affiliation_results)
print('------------')
print('------------')
print(affiliation_results['hits'])
print('------------')
print('------------')
print(affiliation_results['hits']['hits'])
print('------------')
print('------------')
#print(affiliation_results['hits']['hits'].keys())
row1 = affiliation_results['hits']['hits'].pop()
print(row1.keys())
print(affiliation_results['hits']['hits'])
# it is not a flat list, so need to look at hierarchy to see how best to work with it
# at more advanced stage interact with data in a way that keeps the hierarchical structure

#error handling
#if hits total > 0

total_results = requests.get(search_url.url).json()['hits']['total']

#with_tags_percent = (float(with_tags['hits']['total'])/total_results)*100
#missing_tags_percent = (float(missing_tags['hits']['total'])/total_results)*100
affiliation_results_percent = (float(affiliation_results['hits']['total'])/total_results)*100


#print(
#    '{} results out of {}, or {}%, have tags.'.format(
#        with_tags['hits']['total'],
#        total_results,
#        format(with_tags_percent, '.2f')
#    )
#)

#print(
#    '{} results out of {}, or {}%, do NOT have tags.'.format(
#        missing_tags['hits']['total'],
#        total_results,
#        format(missing_tags_percent, '.2f')
#    )
#)

print(
    '{} results out of {}, or {}%, do NOT have tags.'.format(
        affiliation_results['hits']['total'],
        total_results,
        format(affiliation_results_percent, '.2f')
    )
)

print('------------')
print('As a little sanity check....')
print('{} + {} = {}%'.format(with_tags_percent, missing_tags_percent, format(with_tags_percent + missing_tags_percent, '.2f')))

/Users/rjohns14/anaconda/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:821: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/Users/rjohns14/anaconda/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:821: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


{'timed_out': False, '_shards': {'failed': 0, 'total': 5, 'successful': 5}, 'took': 3, 'hits': {'total': 2, 'max_score': 1.0, 'hits': [{'_score': 1.0, '_source': {'date': '2015-06-17T19:41:12+00:00', 'type': 'creativework', 'date_created': '2016-08-19T08:58:01.621579+00:00', 'contributors': [], 'date_published': None, 'is_deleted': False, 'subjects': [], 'institutions': ['University of Oregon Libraries'], 'date_updated': '2015-06-17T19:41:12+00:00', 'date_modified': '2016-08-19T08:58:01.621541+00:00', 'tags': ['libraries', 'program review', 'self-study', 'University Record', 'col_1794_3856', 'com_1794_151', 'com_1794_6373', 'com_1794_3855'], 'title': 'University of Oregon Library System Program Review', 'description': 'University of Oregon Libraries self-study from 1998.', 'sources': ['Scholars Bank University of Oregon'], 'lists': {'institutions': [{'type': 'institution', 'id': 59361, 'name': 'University of Oregon Libraries'}], 'links': [{'type': 'misc', 'url': 'http://hdl.handle.net/

NameError: name 'with_tags_percent' is not defined

In [ ]:
affiliation_results = query_share(search_url.url, affiliation_query)
print(affiliation_results)

In [ ]:
affiliation_results = query_share(search_url.url, affiliation_query)
print(affiliation_results['hits'])

In [ ]:
affiliation_results = query_share(search_url.url, affiliation_query)
print(affiliation_results['hits']['hits'])

In [ ]:
affiliation_results = query_share(search_url.url, affiliation_query)
row1 = affiliation_results['hits']['hits'].pop()
print(row1.keys())

In [ ]:
import os

os.getcwd()
os.chdir('..')
os.getcwd()
os.chdir('share_tutorials')
os.getcwd()

In [ ]:
import csv

with open('names1.csv', 'w') as csvfile:
    fieldnames = ['first_name', 'last_name']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerow({'first_name': 'Baked', 'last_name': 'Beans'})
    writer.writerow({'first_name': 'Lovely', 'last_name': 'Spam'})
    writer.writerow({'first_name': 'Wonderful', 'last_name': 'Spam'})